# A full look into `providers` objects

In [ ]:
import xyzservices.providers as xyz

## What is this "provider" object ?

The provider objects are stored as dictionaries (with attribute access for easy access), so we can explore them in Python (or using tab completion in an interactive session or as a collapsible inventory in Jupyter!):

In [ ]:
xyz

Most of those providers have themselves multiple options. For example, OpenStreetMap provides multiple background styles:

In [ ]:
xyz.OpenStreetMap

Looking at a single one of those options, for example "Mapnik" (the default OpenStreetMap background), we can see this is a `TileProvider` object, which behaves as a dict:

In [ ]:
type(xyz.OpenStreetMap.Mapnik)

We can explore its contents:

In [ ]:
xyz.OpenStreetMap.Mapnik

A "provider object" is then a dictionary with a few required entries, such as the url (required, with `{z}/{x}/{y}` to be replaced), with some additional metadata information (optional). These can be the attribution text to be used on the plot (`attribution`), or the maximum zoom level available for these tiles (`max_zoom`). When this information is available, other libraries like `geopandas` or `contextily` may be smart enough to access and use it automatically.

## Which providers are available by default?

The providers definitions that are shipped with `xyzservices` in the `xyzservices.providers` submodule, are coming from the [`leaflet-providers`](https://github.com/leaflet-extras/leaflet-providers) package, an extension to javascript mapping library Leaflet that contains configurations for various free tile providers. Thus, all providers that are listed on their preview (http://leaflet-extras.github.io/leaflet-providers/preview/) should also be available in `xyzservices`. On top of that, `xyzservices` can have a few more as a bonus.

## Specifying options for a provider

Some providers require additional information, such as an API access key. This can be specified by calling the provider object. Any keyword specified will override the default value in the provider object.

For example, the OpenWeatherMap requires an API key:

In [ ]:
xyz.OpenWeatherMap.Clouds

In [ ]:
xyz.OpenWeatherMap.Clouds.requires_token()

We can specify this API key by calling the object or overriding the attribute:

In [ ]:
# Overriding the attribute will alter existing object
xyz.OpenWeatherMap.Clouds["apiKey"] = "my-private-api-key"

# Calling the object will return a copy
xyz.OpenWeatherMap.Clouds(apiKey="my-private-api-key")

This can then be specified where a key or token is expected. For example:
    
```python
contextily.add_basemap(ax, source=xyz.OpenWeatherMap.Clouds(apiKey="my-private-api-key"))
```


## Overview of built-in providers

Let us create a flat dictionary of the built-in providers:

In [ ]:
providers = {}

def get_providers(provider):
    if "url" in provider:
        providers[provider['name']] = provider
    else:
        for prov in provider.values():
            get_providers(prov)

get_providers(xyz)

This results in quite a few of them already available:

In [ ]:
len(providers)

For this illustration, let's single out the following ones and use `contextily` to have a look at them:

In [ ]:
selection = ['OpenStreetMap.Mapnik', 
             'OpenTopoMap', 
             'Stamen.Toner', 
             'Stamen.TonerLite', 
             'Stamen.Terrain', 
             'Stamen.TerrainBackground', 
             'Stamen.Watercolor', 
             'NASAGIBS.ViirsEarthAtNight2012', 
             'CartoDB.Positron', 
             'CartoDB.Voyager'
            ]

In [ ]:
import matplotlib.pyplot as plt
import contextily

# we are using an extent around Mexico for the examples
extent = (-12600000, -10300000, 1800000, 3800000)

fig, axs = plt.subplots(2, 5, figsize=(25, 10))
axs = axs.flatten()
for name, ax in zip(selection, axs):
    ax.axis(extent)
    contextily.add_basemap(ax, source=providers[name])
    ax.set_title(name)
    ax.set_axis_off()
plt.show()